In [125]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.naive_bayes import MultinomialNB
import nltk
import matplotlib
from matplotlib import pyplot as plt
import string

In [148]:
train=pd.read_csv('train.csv')

In [149]:
train_count=train.groupby(['user.id'])["user.id"].size().reset_index(name='Count')
train_count=train_count['Count']

In [150]:
#train=train.groupby(['user.id','age'])["text"].apply(lambda x: "%s" % ' '.join(x)).reset_index(name='text')

In [151]:
train_id=train['user.id']

In [152]:
train.head(10)

,post.id,user.id,gender,topic,sign,date,text,age
0,1,11869,male,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",15
1,2,11869,male,Student,Leo,"13,May,2004",These are the team members: Drewe...,15
2,3,11869,male,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,15
3,4,11869,male,Student,Leo,"12,May,2004",testing!!! testing!!!,15
4,5,16332,male,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,33
5,6,16332,male,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...,33
6,7,16332,male,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...,33
7,8,16332,male,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o...",33
8,9,16332,male,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article ...,33
9,10,16332,male,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a l...,33


In [61]:
def count_punct(text):
    count = lambda l1,l2: sum([1 for x in l1 if x in l2])
    s=count(text,set(string.punctuation))                                                                                                      
    return s

In [62]:
train['punct_count']=train['text'].apply(count_punct)
train['punct_count']=train.iloc[:,3].div(train_count,axis=0)
train['punct_count']=train.iloc[:,3].div(train.iloc[:,3].max(),axis=0)

In [63]:
train.head()

,user.id,age,text,punct_count
0,1,17,Doritos Fuel of Space urlLink : Ian My...,0.003312
1,3,16,"Yeah, I set up this blog so Heather w...",0.022772
2,4,17,Not too much exciting has happened ...,0.025595
3,5,47,European Pilgrimage For High sch...,0.017341
4,6,16,Last night's dream is very blurry. T...,0.015028


In [10]:
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)
train['text'] = train['text'].apply(remove_punctuation)

In [11]:
nltk.download('stopwords')
sw = stopwords.words('english')
def stopwords(text):
    '''a function for removing the stopword'''
    # removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)
train['text'] = train['text'].apply(stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
stemmer = SnowballStemmer("english")

def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text) 
train['text'] = train['text'].apply(stemming)
train.head(10)

,user.id,age,text,punct_count
0,1,17,dorito fuel space urllink ian myer nasa new pl...,0.003312
1,3,16,yeah set blog heather would happysh better lol...,0.022772
2,4,17,much excit happen weekend ive realli busi ive ...,0.025595
3,5,47,european pilgrimag high school student st mich...,0.017341
4,6,16,last night dream blurri detail rememb patti qu...,0.015028
5,7,25,june 21 2004 spain v portug good euro portug i...,0.008331
6,12,23,well 1 day left less 36 hour presenc favorit b...,0.019151
7,14,37,everi see post one forum goe someth like “i pl...,0.016922
8,15,17,well around 3 adam jake went outsid smoke pipe...,0.015714
9,16,15,yesterdayw wif sister meet huiqingr quit enjoy...,0.015392


In [13]:
train.to_csv("train_clean.csv", encoding = 'utf-8', index = False)

In [104]:
train=pd.read_csv("train_clean.csv")

In [106]:
train=train.drop(['punct_count'],axis=1)

In [153]:
#new
count_vect = CountVectorizer(max_features=5500)
X_train_counts = count_vect.fit_transform(train.text)
X_train_counts.shape

(442961, 5500)

In [154]:
#new
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(442961, 5500)

In [155]:
train=train.drop(['user.id','text'],axis=1)

In [156]:
tfidf_df = pd.DataFrame(X_train_tfidf.todense())

MemoryError: 

In [134]:
train_total=train.join(tfidf_df,how='right')

In [135]:
train_total.head()

,age,0,1,2,3,4,5,6,7,8,...,5490,5491,5492,5493,5494,5495,5496,5497,5498,5499
0,17,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000
1,16,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000
2,17,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.051162,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000
3,47,0.016926,0.002959,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.006591,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000
4,16,0.008680,0.000246,0.003211,0.00287,0.002719,0.003318,0.002057,0.003716,0.000733,...,0.012059,0.000358,0.0,0.000695,0.00095,0.0,0.000934,0.0,0.000627,0.001092


In [136]:
train_age=train_total['age']

In [137]:
train_total=train_total.drop('age',axis=1)

In [ ]:
clf = MultinomialNB().fit(train_total, train_age)

In [46]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(train_total, train_age)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
test=pd.read_csv('test.csv')

In [ ]:
test_count=test.groupby(['user.id'])["user.id"].size().reset_index(name='Count')
test_count=test_count['Count']
#test=test.groupby('user.id')["text"].apply(lambda x: "%s" % ' '.join(x)).reset_index(name='text')
test_id=test['user.id']
#test['punct_count']=test['text'].apply(count_punct)
#test['punct_count']=test.iloc[:,2].div(test_count,axis=0)
#test['punct_count']=test.iloc[:,2].div(test.iloc[:,2].max(),axis=0)
#test.head()


In [29]:
test['text'] = test['text'].apply(stopwords)
test['text'] = test['text'].apply(stemming)
test.head(10)

,user.id,text,punct_count
0,2,final saw nathan tattoo tonight. like it! fact...,0.001959
1,8,today last day shortest. one session last 10:0...,0.003216
2,9,hell internet? 2:16am song: exact i'm at - wee...,0.010679
3,10,hotel&nbsp; urllink urllink thrid & win goal&n...,0.007363
4,11,last night receiv messag friend minnesota. tol...,0.005848
5,13,"hey every1 came back school fun, think subject...",0.003839
6,20,chang blogspot address...mi origin reason want...,0.006626
7,26,m33ting chrome sunlight > line say all. online...,0.005621
8,27,i'm mental prepar journi western psyche. told ...,0.003361
9,29,"hrm. i'v decid mu calori intake, up amount exc...",0.010586


In [ ]:
test.to_csv("test_clean.csv", encoding = 'utf-8', index = False)

In [116]:
test=pd.read_csv("test_clean.csv")

In [ ]:
test=test.drop(['punct_count'],axis=1)

In [ ]:
X_test_counts = count_vect.fit_transform(test.text)
X_test_counts.shape
tfidf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
X_test_tfidf.shape

In [ ]:
test_2=test

In [ ]:
test=test.drop(['user.id','text'],axis=1)
tfidf_df = pd.DataFrame(X_test_tfidf.todense())
test_total=test.join(tfidf_df,how='right')
test_total.head()

In [ ]:
#predicted = clf.predict(test_total)

In [ ]:
predicted = clf.predict(test_total)

In [ ]:
cols = ["user.id", "age"]
test_output = pd.DataFrame(columns = cols)
test_output[["user.id"]] = test_2[["user.id"]]
test_output[["age"]] = predicted

In [ ]:
test_output.head(20)

In [42]:
test_output['age']=test_output['age'].apply(np.floor)

In [44]:
# train.to_csv("train_FRE.csv", encoding = 'utf-8', index = False)
test_output.to_csv("test_sklearn_output.csv", encoding = 'utf-8', index = False)

In [ ]:
#def length(text):    
 #   '''a function which returns the length of text'''
 #   return len(text)

In [ ]:
#train['length'] = train['text'].apply(length)
#train.head(10)

In [ ]:
# create the object of tfid vectorizer
tfid_vectorizer = TfidfVectorizer("english")
# fit the vectorizer using the text data
tfid_vectorizer.fit(train['text'])
# collect the vocabulary items used in the vectorizer
dictionary = tfid_vectorizer.vocabulary_.items()  

In [ ]:
train.to_csv("train_sklearn_intermediate.csv", encoding = 'utf-8', index = False)

In [ ]:
tfid_matrix = tfid_vectorizer.transform(train['text'])
#collect the tfid matrix in numpy array

In [ ]:
tfid_matrix